In [3]:
import torch
from sympy.physics.vector.printing import params
from torch import nn
import d2l

In [4]:
# 这个块包含一个多层感知机，其具有256个隐藏单元的隐藏层和一个10维输出层。
# 该MLP类继承了表示块的类。我们的实现只需要提供我们自己的构造函数和前向传播函数即可
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        # 首先调用父类 Module 的构造函数来执行必要的初始化
        super(MLP, self).__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

        # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [5]:
class MySequential(nn.Module):
    """
    接下来考虑顺序块 Sequential 类是如何工作的
    Sequential 的设计是为了把其他模块串起来。为了构建我们自己的简化的 MySequential，我们需要定义两个函数：
    1. 一种将块逐个追加到列表中的函数；
    2. 一种前向传播函数，用于将输入按追加的顺序传递给块组成“链条”
    """
    def __init__(selfself, *args):
        Super().__init__()
        for idx, module in enumerate(args):
            # 这里， module 是 Module 子类的一个实例。我们把它保存在 Module 类的成员变量 _modules 中。_module的类型是 OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict 保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

In [6]:
# 自定义 Module 并实现 forward 函数
class FixedHiddenMLP(nn.Module):
    def __init__(self, input_size):
        super(FixedHiddenMLP, self).__init__()
        # 不计算梯度的随机权重参数，在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(input_size, input_size)

    def forward(selfself, X):
        X = self.linear(X)
        # 使用创建的常量参数以及 relu 和 mm 函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [21]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8,1))
X = torch.rand(size=(2,4))
y = net(X)

In [22]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.2649, -0.2580,  0.1591,  0.1057, -0.1232,  0.0184, -0.1005,  0.3013]])), ('bias', tensor([0.2092]))])


In [23]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.2092], requires_grad=True)
tensor([0.2092])


In [24]:
net[2].weight.grad == None

True

In [25]:
print(*[(name, param.shape) for name, param in net.named_parameters()])

('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [26]:
net.state_dict()["2.bias"].data

tensor([0.2092])

In [34]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f"block{i}", block1())
    return net

regnet = nn.Sequential(block2(), nn.Linear(4,1))
regnet(X)

tensor([[-0.1216],
        [-0.1216]], grad_fn=<AddmmBackward0>)

In [35]:
print(regnet)

Sequential(
  (0): Sequential(
    (block0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [36]:
regnet[0][1][0].bias.data

tensor([ 0.3782, -0.1245, -0.3083, -0.3703,  0.2460,  0.0506,  0.3627, -0.4484])

In [37]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)

net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0062,  0.0044,  0.0095,  0.0020]), tensor(0.))

In [39]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)

net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [40]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)

def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.4689, -0.5319, -0.3076, -0.4791])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [44]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
print(net[0].weight[:2])

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])
tensor([[-8.6139, -0.0000, -8.1797,  8.2329],
        [ 8.3309, -8.4074, -0.0000, -0.0000]], grad_fn=<SliceBackward0>)


In [48]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data

tensor([[42.0000,  3.0000, -5.1797, 11.2329],
        [11.3309, -5.4074,  3.0000,  3.0000],
        [12.0012,  3.0000,  3.0000,  3.0000],
        [ 3.0000, 12.2128, -5.7434,  3.0000],
        [ 3.0000,  9.3810, 12.7544,  9.6391],
        [ 3.0000,  9.1548,  3.0000, 12.9924],
        [11.3120,  3.0000, -5.1478, -4.8658],
        [-6.4457,  3.0000, 11.0230, -2.5043]])

In [49]:
# 有时我们希望在多个层间共享参数：我们可以定义一个稠密层，然后使用它的参数来设置另一个层的参数。
# 我们需要给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))

net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
# 确保它们实际上是同一个对象，而不是只有相同的值
net[2].weight.data[0,0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


In [53]:
import torch
import torch.nn.functional as F
from torch import nn

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

In [54]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [55]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

In [56]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(-3.7253e-09, grad_fn=<MeanBackward0>)

In [57]:
# 自定义版本的全连接层
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        """
        :param in_units: 输入数
        :param units:  输出数
        两个参数共同确定了全连接层的维度
        """
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))

    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [58]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[ 0.6505, -1.9611,  0.6972],
        [-0.8842,  0.0967,  1.2107],
        [-1.1519,  0.3771, -0.6761],
        [ 1.6588,  0.7082, -0.3434],
        [-1.0368, -1.1241, -2.0085]], requires_grad=True)

In [62]:
linear(torch.rand(2,5))

tensor([[0.3808, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000]])

In [64]:
import torch
from torch import nn
from torch.nn import functional as F

In [65]:
x = torch.arange(4)
torch.save(x, "x-file")
x2 = torch.load("x-file")
x2

tensor([0, 1, 2, 3])

In [66]:
y = torch.zeros(4)
torch.save([x,y], 'x-files')
x2, y2 = torch.load("x-files")
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [67]:
mydict = {'x':x, 'y':y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

## 加载和保存模型参数

In [69]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, X):
        return self.output(F.relu(self.hidden(X)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

In [70]:
torch.save(net.state_dict(), "mlp.params")

In [71]:
clone = MLP()
clone.load_state_dict(torch.load("mlp.params"))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [73]:
# 两模型虽不是同一个模型，但两模型所表示的函数是相同的
# 因此对相同的输入 X，它们的输出也应该是相同的
Y_clone = clone(X)
Y == Y_clone

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])